In [1]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import *
import vgg16
import pandas as pd
import numpy as np
from glob import glob
from keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import log_loss
%matplotlib inline

data_dir = os.path.join(os.getcwd(), 'data', 'whale')
img_dir  = os.path.join(os.getcwd(), 'data', 'whale', 'imgs')
batch_size=64
num_class = 447
img_shape=(3, 224, 224)
target_size = (224,224)

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [37]:
data_dir

'/home/ubuntu/courses/deeplearning1/nbs/data/whale'

In [38]:
img_dir

'/home/ubuntu/courses/deeplearning1/nbs/data/whale/imgs'

In [85]:
def get_tr_batches(dr = img_dir+ '/train', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=True, target_size=target_size, batch_size=64)

def get_val_batches(dr = img_dir+ '/valid', gen = ImageDataGenerator(), batch_size=batch_size):
    return gen.flow_from_directory(dr, shuffle=False, target_size=target_size, batch_size=batch_size)

In [23]:
tr_features = load_array('vgg_conv_tr.bc')
val_features = load_array('vgg_val_tr.bc')


In [26]:
tr_labels = load_array('whale_tr_classes.bc')
val_labels = load_array('whale_val_classes.bc')


In [41]:
def fit_model(model, epochs, tr_b = get_tr_batches(), val_b = get_val_batches()):
    model.fit_generator(tr_b, tr_b.nb_sample, 
                    validation_data=val_b, 
                    nb_val_samples=val_b.nb_sample, 
                    nb_epoch=epochs)

Found 3635 images belonging to 447 classes.
Found 909 images belonging to 447 classes.


In [92]:
vgg = Vgg16()

In [93]:
model = vgg.model

In [94]:
last_conv = [i for i, v in enumerate(vgg.model.layers) if type(v) is Conv2D][-1]
conv_layers = vgg.model.layers[:last_conv+1] 
fc_layers = vgg.model.layers[last_conv+1:]

In [95]:
def get_fc_model():
    model = Sequential([
            MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
            Flatten(),
            Dense(4096, activation='relu'),
            Dropout(0.5),
            Dense(4096, activation='relu'),
            Dropout(0.5),
            Dense(1000, activation='softmax')
        ])
    
    for l1, l2, in zip(model.layers, fc_layers): l1.set_weights(l2.get_weights())
    return model

In [96]:
fc_model = get_fc_model()

In [97]:
fc_model.layers = fc_model.layers[:-1]

In [98]:
for l in fc_model.layers: l.trainable = False

In [99]:
fc_model.add(Dense(num_class, activation='softmax'))

In [100]:
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_18 (MaxPooling2D)   (None, 512, 7, 7)     0           maxpooling2d_input_7[0][0]       
____________________________________________________________________________________________________
flatten_9 (Flatten)              (None, 25088)         0           maxpooling2d_18[0][0]            
____________________________________________________________________________________________________
dense_29 (Dense)                 (None, 4096)          0           flatten_9[0][0]                  
____________________________________________________________________________________________________
dropout_17 (Dropout)             (None, 4096)          0           dense_29[0][0]                   
___________________________________________________________________________________________

In [101]:
fc_model.compile(Adam(lr=0.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
fc_model.fit(tr_features, tr_labels,batch_size=batch_size, validation_data=(val_features, val_labels), nb_epoch=300)

Train on 3635 samples, validate on 909 samples
Epoch 1/300
3635/3635 [==============================] - 2s - loss: 6.1020 - acc: 0.0025 - val_loss: 6.1013 - val_acc: 0.0055
Epoch 2/300
3635/3635 [==============================] - 2s - loss: 6.0977 - acc: 0.0080 - val_loss: 6.0987 - val_acc: 0.0055
Epoch 3/300
1024/3635 [=======>......................] - ETA: 1s - loss: 6.0943 - acc: 0.0107

In [34]:
fc_model.fit(tr_features, tr_labels,batch_size=batch_size, validation_data=(val_features, val_labels), nb_epoch=5)

Train on 3635 samples, validate on 909 samples
Epoch 1/5
3635/3635 [==============================] - 2s - loss: 6.0970 - acc: 0.0085 - val_loss: 6.0973 - val_acc: 0.0055
Epoch 2/5
3635/3635 [==============================] - 2s - loss: 6.0927 - acc: 0.0102 - val_loss: 6.0951 - val_acc: 0.0055
Epoch 3/5
3635/3635 [==============================] - 2s - loss: 6.0892 - acc: 0.0077 - val_loss: 6.0931 - val_acc: 0.0055
Epoch 4/5
3635/3635 [==============================] - 2s - loss: 6.0854 - acc: 0.0091 - val_loss: 6.0917 - val_acc: 0.0044
Epoch 5/5
3635/3635 [==============================] - 2s - loss: 6.0816 - acc: 0.0099 - val_loss: 6.0903 - val_acc: 0.0033


In [36]:
fc_model.lr = 0.001

In [35]:
fc_model.fit(tr_features, tr_labels,batch_size=batch_size, validation_data=(val_features, val_labels), nb_epoch=10)

Train on 3635 samples, validate on 909 samples
Epoch 1/10
3635/3635 [==============================] - 2s - loss: 6.0774 - acc: 0.0096 - val_loss: 6.0888 - val_acc: 0.0066
Epoch 2/10
3635/3635 [==============================] - 2s - loss: 6.0737 - acc: 0.0091 - val_loss: 6.0872 - val_acc: 0.0077
Epoch 3/10
3635/3635 [==============================] - 2s - loss: 6.0696 - acc: 0.0107 - val_loss: 6.0851 - val_acc: 0.0055
Epoch 4/10
3635/3635 [==============================] - 2s - loss: 6.0653 - acc: 0.0099 - val_loss: 6.0834 - val_acc: 0.0055
Epoch 5/10
3635/3635 [==============================] - 2s - loss: 6.0614 - acc: 0.0116 - val_loss: 6.0817 - val_acc: 0.0066
Epoch 6/10
3635/3635 [==============================] - 2s - loss: 6.0575 - acc: 0.0121 - val_loss: 6.0800 - val_acc: 0.0055
Epoch 7/10
3635/3635 [==============================] - 2s - loss: 6.0535 - acc: 0.0118 - val_loss: 6.0783 - val_acc: 0.0055
Epoch 8/10
3635/3635 [==============================] - 2s - loss: 6.0504 - ac

In [38]:
fc_model.fit(tr_features, tr_labels,batch_size=batch_size, validation_data=(val_features, val_labels), nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 6.0081 - acc: 0.0146 - val_loss: 6.0612 - val_acc: 0.0088
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 6.0048 - acc: 0.0146 - val_loss: 6.0604 - val_acc: 0.0088
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 6.0018 - acc: 0.0140 - val_loss: 6.0593 - val_acc: 0.0088
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.9985 - acc: 0.0143 - val_loss: 6.0579 - val_acc: 0.0088
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.9953 - acc: 0.0146 - val_loss: 6.0572 - val_acc: 0.0088
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.9926 - acc: 0.0165 - val_loss: 6.0562 - val_acc: 0.0088
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.9903 - acc: 0.0151 - val_loss: 6.0553 - val_acc: 0.0088
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.9865 - ac

In [39]:
cfc_model.fit(tr_features, tr_labels,batch_size=batch_size, validation_data=(val_features, val_labels), nb_epoch=50)`

Train on 3635 samples, validate on 909 samples
Epoch 1/50
3635/3635 [==============================] - 2s - loss: 5.9381 - acc: 0.0146 - val_loss: 6.0393 - val_acc: 0.0099
Epoch 2/50
3635/3635 [==============================] - 2s - loss: 5.9360 - acc: 0.0149 - val_loss: 6.0392 - val_acc: 0.0110
Epoch 3/50
3635/3635 [==============================] - 2s - loss: 5.9330 - acc: 0.0165 - val_loss: 6.0375 - val_acc: 0.0099
Epoch 4/50
3635/3635 [==============================] - 2s - loss: 5.9309 - acc: 0.0143 - val_loss: 6.0381 - val_acc: 0.0099
Epoch 5/50
3635/3635 [==============================] - 2s - loss: 5.9280 - acc: 0.0162 - val_loss: 6.0370 - val_acc: 0.0099
Epoch 6/50
3635/3635 [==============================] - 2s - loss: 5.9260 - acc: 0.0151 - val_loss: 6.0363 - val_acc: 0.0099
Epoch 7/50
3635/3635 [==============================] - 2s - loss: 5.9238 - acc: 0.0157 - val_loss: 6.0355 - val_acc: 0.0099
Epoch 8/50
3635/3635 [==============================] - 2s - loss: 5.9215 - ac

# Train all FC layers

In [49]:
fc_model = get_fc_model()

In [50]:
fc_model.layers = fc_model.layers[:-1]

In [51]:
fc_model.add(Dense(num_class, activation='softmax'))

In [52]:
fc_model.compile(RMSprop(lr=0.00001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [53]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=3)

Train on 3635 samples, validate on 909 samples
Epoch 1/3
3635/3635 [==============================] - 6s - loss: 6.1025 - acc: 0.0022 - val_loss: 6.1018 - val_acc: 0.0011
Epoch 2/3
3635/3635 [==============================] - 6s - loss: 6.0999 - acc: 0.0039 - val_loss: 6.1022 - val_acc: 0.0044
Epoch 3/3
3635/3635 [==============================] - 6s - loss: 6.0973 - acc: 0.0063 - val_loss: 6.1023 - val_acc: 0.0055


In [54]:
fc_model.optimizer.lr = 0.001

In [55]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=3)

Train on 3635 samples, validate on 909 samples
Epoch 1/3
3635/3635 [==============================] - 6s - loss: 6.0951 - acc: 0.0074 - val_loss: 6.1022 - val_acc: 0.0044
Epoch 2/3
3635/3635 [==============================] - 6s - loss: 6.0926 - acc: 0.0083 - val_loss: 6.1018 - val_acc: 0.0055
Epoch 3/3
3635/3635 [==============================] - 6s - loss: 6.0904 - acc: 0.0096 - val_loss: 6.1017 - val_acc: 0.0033


In [56]:
fc_model.optimizer.lr = 0.01

In [57]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=10)

Train on 3635 samples, validate on 909 samples
Epoch 1/10
3635/3635 [==============================] - 6s - loss: 6.0879 - acc: 0.0102 - val_loss: 6.1021 - val_acc: 0.0033
Epoch 2/10
3635/3635 [==============================] - 6s - loss: 6.0854 - acc: 0.0116 - val_loss: 6.1022 - val_acc: 0.0033
Epoch 3/10
3635/3635 [==============================] - 6s - loss: 6.0834 - acc: 0.0127 - val_loss: 6.1013 - val_acc: 0.0033
Epoch 4/10
3635/3635 [==============================] - 6s - loss: 6.0815 - acc: 0.0135 - val_loss: 6.1013 - val_acc: 0.0022
Epoch 5/10
3635/3635 [==============================] - 6s - loss: 6.0792 - acc: 0.0160 - val_loss: 6.1013 - val_acc: 0.0022
Epoch 6/10
3635/3635 [==============================] - 6s - loss: 6.0773 - acc: 0.0160 - val_loss: 6.1010 - val_acc: 0.0044
Epoch 7/10
3635/3635 [==============================] - 6s - loss: 6.0752 - acc: 0.0168 - val_loss: 6.1009 - val_acc: 0.0044
Epoch 8/10
3635/3635 [==============================] - 6s - loss: 6.0732 - ac

# Less Dropout

In [58]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]

In [59]:
def get_fc_model():
    model = Sequential([
            MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
            Flatten(),
            Dense(4096, activation='relu'),
            Dropout(0.),
            Dense(4096, activation='relu'),
            Dropout(0.),
            Dense(1000, activation='softmax')
        ])
    
    for l1, l2, in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
    return model

In [60]:
fc_model = get_fc_model()

In [61]:
fc_model.layers = fc_model.layers[:-1]

In [62]:
for l in fc_model.layers: l.trainable = False

In [63]:
fc_model.add(Dense(num_class, activation='softmax'))

In [64]:
fc_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
maxpooling2d_12 (MaxPooling2D)   (None, 512, 7, 7)     0           maxpooling2d_input_6[0][0]       
____________________________________________________________________________________________________
flatten_7 (Flatten)              (None, 25088)         0           maxpooling2d_12[0][0]            
____________________________________________________________________________________________________
dense_22 (Dense)                 (None, 4096)          0           flatten_7[0][0]                  
____________________________________________________________________________________________________
dropout_13 (Dropout)             (None, 4096)          0           dense_22[0][0]                   
___________________________________________________________________________________________

In [65]:
fc_model.compile(Adam(lr=0.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [66]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=5)

Train on 3635 samples, validate on 909 samples
Epoch 1/5
3635/3635 [==============================] - 2s - loss: 6.1014 - acc: 0.0044 - val_loss: 6.1001 - val_acc: 0.0154
Epoch 2/5
3635/3635 [==============================] - 2s - loss: 6.0976 - acc: 0.0085 - val_loss: 6.0966 - val_acc: 0.0066
Epoch 3/5
3635/3635 [==============================] - 2s - loss: 6.0928 - acc: 0.0072 - val_loss: 6.0926 - val_acc: 0.0022
Epoch 4/5
3635/3635 [==============================] - 2s - loss: 6.0889 - acc: 0.0072 - val_loss: 6.0896 - val_acc: 0.0022
Epoch 5/5
3635/3635 [==============================] - 2s - loss: 6.0858 - acc: 0.0072 - val_loss: 6.0875 - val_acc: 0.0022


In [67]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 6.0832 - acc: 0.0072 - val_loss: 6.0856 - val_acc: 0.0022
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 6.0808 - acc: 0.0072 - val_loss: 6.0839 - val_acc: 0.0022
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 6.0785 - acc: 0.0072 - val_loss: 6.0823 - val_acc: 0.0022
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 6.0762 - acc: 0.0072 - val_loss: 6.0809 - val_acc: 0.0022
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 6.0740 - acc: 0.0072 - val_loss: 6.0794 - val_acc: 0.0022
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 6.0718 - acc: 0.0072 - val_loss: 6.0782 - val_acc: 0.0022
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 6.0695 - acc: 0.0077 - val_loss: 6.0768 - val_acc: 0.0022
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 6.0672 - ac

In [68]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 6.0309 - acc: 0.0105 - val_loss: 6.0521 - val_acc: 0.0099
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 6.0291 - acc: 0.0105 - val_loss: 6.0513 - val_acc: 0.0099
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 6.0275 - acc: 0.0105 - val_loss: 6.0500 - val_acc: 0.0099
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 6.0258 - acc: 0.0105 - val_loss: 6.0489 - val_acc: 0.0099
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 6.0242 - acc: 0.0105 - val_loss: 6.0479 - val_acc: 0.0099
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 6.0226 - acc: 0.0105 - val_loss: 6.0469 - val_acc: 0.0099
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 6.0210 - acc: 0.0105 - val_loss: 6.0459 - val_acc: 0.0099
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 6.0194 - ac

In [69]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 5.9934 - acc: 0.0105 - val_loss: 6.0301 - val_acc: 0.0099
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 5.9920 - acc: 0.0105 - val_loss: 6.0292 - val_acc: 0.0099
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 5.9907 - acc: 0.0105 - val_loss: 6.0284 - val_acc: 0.0099
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.9893 - acc: 0.0105 - val_loss: 6.0274 - val_acc: 0.0099
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.9880 - acc: 0.0105 - val_loss: 6.0271 - val_acc: 0.0099
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.9868 - acc: 0.0105 - val_loss: 6.0259 - val_acc: 0.0099
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.9854 - acc: 0.0105 - val_loss: 6.0259 - val_acc: 0.0099
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.9841 - ac

In [70]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 5.9621 - acc: 0.0105 - val_loss: 6.0142 - val_acc: 0.0099
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 5.9610 - acc: 0.0105 - val_loss: 6.0136 - val_acc: 0.0099
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 5.9599 - acc: 0.0105 - val_loss: 6.0134 - val_acc: 0.0099
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.9587 - acc: 0.0105 - val_loss: 6.0127 - val_acc: 0.0099
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.9576 - acc: 0.0105 - val_loss: 6.0123 - val_acc: 0.0099
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.9565 - acc: 0.0105 - val_loss: 6.0118 - val_acc: 0.0099
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.9553 - acc: 0.0105 - val_loss: 6.0116 - val_acc: 0.0099
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.9542 - ac

In [71]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 5.9351 - acc: 0.0105 - val_loss: 6.0032 - val_acc: 0.0099
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 5.9341 - acc: 0.0105 - val_loss: 6.0028 - val_acc: 0.0099
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 5.9331 - acc: 0.0105 - val_loss: 6.0026 - val_acc: 0.0099
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.9321 - acc: 0.0105 - val_loss: 6.0026 - val_acc: 0.0099
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.9312 - acc: 0.0105 - val_loss: 6.0022 - val_acc: 0.0099
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.9301 - acc: 0.0105 - val_loss: 6.0020 - val_acc: 0.0099
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.9294 - acc: 0.0105 - val_loss: 6.0013 - val_acc: 0.0099
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.9283 - ac

In [72]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 5.9117 - acc: 0.0105 - val_loss: 5.9962 - val_acc: 0.0099
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 5.9108 - acc: 0.0105 - val_loss: 5.9957 - val_acc: 0.0099
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 5.9100 - acc: 0.0105 - val_loss: 5.9956 - val_acc: 0.0099
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.9091 - acc: 0.0105 - val_loss: 5.9951 - val_acc: 0.0099
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.9082 - acc: 0.0105 - val_loss: 5.9953 - val_acc: 0.0099
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.9074 - acc: 0.0105 - val_loss: 5.9949 - val_acc: 0.0099
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.9066 - acc: 0.0105 - val_loss: 5.9950 - val_acc: 0.0099
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.9057 - ac

In [73]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 5.8913 - acc: 0.0105 - val_loss: 5.9921 - val_acc: 0.0099
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 5.8906 - acc: 0.0105 - val_loss: 5.9917 - val_acc: 0.0099
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 5.8898 - acc: 0.0105 - val_loss: 5.9916 - val_acc: 0.0099
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.8891 - acc: 0.0105 - val_loss: 5.9914 - val_acc: 0.0099
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.8884 - acc: 0.0105 - val_loss: 5.9914 - val_acc: 0.0099
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.8876 - acc: 0.0105 - val_loss: 5.9911 - val_acc: 0.0099
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.8869 - acc: 0.0105 - val_loss: 5.9910 - val_acc: 0.0099
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.8861 - ac

In [74]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 5.8735 - acc: 0.0105 - val_loss: 5.9903 - val_acc: 0.0099
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 5.8729 - acc: 0.0105 - val_loss: 5.9902 - val_acc: 0.0099
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 5.8722 - acc: 0.0105 - val_loss: 5.9903 - val_acc: 0.0099
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.8717 - acc: 0.0105 - val_loss: 5.9898 - val_acc: 0.0099
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.8710 - acc: 0.0105 - val_loss: 5.9902 - val_acc: 0.0099
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.8703 - acc: 0.0105 - val_loss: 5.9901 - val_acc: 0.0099
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.8697 - acc: 0.0105 - val_loss: 5.9901 - val_acc: 0.0099
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.8689 - ac

In [76]:
fc_model.optimizer.lr = 0.00001

In [77]:
fc_model.fit(tr_features, 
             tr_labels,
             batch_size=batch_size, 
             validation_data=(val_features, val_labels), 
             nb_epoch=25)

Train on 3635 samples, validate on 909 samples
Epoch 1/25
3635/3635 [==============================] - 2s - loss: 5.8579 - acc: 0.0107 - val_loss: 5.9907 - val_acc: 0.0121
Epoch 2/25
3635/3635 [==============================] - 2s - loss: 5.8574 - acc: 0.0107 - val_loss: 5.9906 - val_acc: 0.0121
Epoch 3/25
3635/3635 [==============================] - 2s - loss: 5.8568 - acc: 0.0107 - val_loss: 5.9907 - val_acc: 0.0121
Epoch 4/25
3635/3635 [==============================] - 2s - loss: 5.8562 - acc: 0.0107 - val_loss: 5.9906 - val_acc: 0.0121
Epoch 5/25
3635/3635 [==============================] - 2s - loss: 5.8556 - acc: 0.0107 - val_loss: 5.9905 - val_acc: 0.0121
Epoch 6/25
3635/3635 [==============================] - 2s - loss: 5.8551 - acc: 0.0107 - val_loss: 5.9907 - val_acc: 0.0121
Epoch 7/25
3635/3635 [==============================] - 2s - loss: 5.8545 - acc: 0.0105 - val_loss: 5.9906 - val_acc: 0.0121
Epoch 8/25
3635/3635 [==============================] - 2s - loss: 5.8539 - ac

In [78]:
test_features = load_array('vgg_conv_test.bc')

In [79]:
fc_model.predict?

In [80]:
preds = fc_model.predict(test_features)

In [82]:
from create_whale_sub import create_sub

In [86]:
test_batches = ImageDataGenerator().flow_from_directory(img_dir+'/test', shuffle=False)
tr_batches = ImageDataGenerator().flow_from_directory(img_dir+'/train', shuffle=False)

Found 6925 images belonging to 1 classes.
Found 3635 images belonging to 447 classes.


In [84]:
def create_sub(preds, test_batches, train_batches):
    file_names = np.array([f[f.find('/') +1:] for f in test_batches.filenames])
    
    idx_class = [(v, k) for k,v in tr_batches.class_indices.items()]
    idx_class= sorted(idx_class, key=lambda x: x[0])
    classes = [x[1] for x in idx_class]
    data = np.hstack((file_names[:, np.newaxis], preds))
    columns = ['Image'] + classes
    return pd.DataFrame(data, columns=columns)

In [87]:
sub = create_sub(preds, test_batches, tr_batches)

In [89]:
len(sub)

6925

In [90]:
sub.to_csv('vgg_train_ll.csv', index=False)

In [91]:
from IPython.display import FileLink
FileLink('vgg_train_ll.csv')

/home/ubuntu/courses/deeplearning1/nbs/vgg_train_ll.csv

scores 5.99777

In [44]:
def calc_loss(model, val_batches, classes):
    preds = model.predict_generator(val_batches, val_batches.nb_sample)
    clipped_preds = np.maximum(preds, 0.00022)
    #log_loss(val_b.classes, preds, labels=range(447))
    print('raw val loss: {}'.format(log_loss(classes, preds, labels=range(447))))
    print('clipped val loss: {}'.format(log_loss(classes, clipped_preds, labels=range(447))))

In [45]:
class LossCallback(keras.callbacks.Callback):
    
    def __init__(self):
        self.val_batches = get_val_batches()
        self.classes = self.val_batches.classes
        #self.n_val = self.val_batches.nb_sample
    
    def on_epoch_end(self, epoch, logs={}):
        calc_loss(self.model, self.val_batches, self.classes) 
    

In [46]:
loss_cb = LossCallback()

Found 909 images belonging to 447 classes.
